<a href="https://colab.research.google.com/github/sweetbaer03/KOR_Sentiment-Analysis/blob/main/NSMC_KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NSMC Sentiment Analysis**


KoBERT 모델 사용 

참고소스 :
https://github.com/SKTBrain/KoBERT

https://github.com/SKTBrain/KoBERT#using-with-pytorch

https://colab.research.google.com/github/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

In [1]:
 !pip install mxnet
 !pip install gluonnlp pandas tqdm
 !pip install sentencepiece
!pip3 install transformers==3.5.0

     |████████████████████████████████| 55.0MB 103kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 13.7MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588513 sha256=e99a86d12e62604e32182f42d5510cc5aaba932cd4be584b9aa530d54d7734a3
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 15.2MB/s 
     |████████████████████████████████| 1.3MB 12.0MB/s 
     |████████████████████████████████| 2.9MB 56.0MB/s 
     |████████████████████████████████| 1.1MB 56.9MB/s 
     |████████████████████████████████| 890kB 58.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=212a92bf4d3de98bb68614dfe63fc6629e81916c1499c5000cc95282e9930f10
  Stored in direc

In [2]:
 !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-6qmnsl7n
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-6qmnsl7n
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=22a2ad468e2daf3a362e3eb498c09b567f56dd5fd13cf7585df7affeeb3af492
  Stored in directory: /tmp/pip-ephem-wheel-cache-fep8e_7i/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [3]:
import gluonnlp as nlp
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import csv
import os
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from tensorboardcolab import *
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# 1. KoBERT 불러오기

In [4]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [5]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


# 2. 데이터 로드

github에서 필요한 데이터파일 다운

In [6]:
!git clone https://github.com/sweetbaer03/KOR_Sentiment-Analysis.git

Cloning into 'KOR_Sentiment-Analysis'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 2), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (15/15), done.


# 3. 전처리

## 3.1 NSMC

In [7]:
# train/test set 데이터 로드
train_data = pd.read_csv("KOR_Sentiment-Analysis/data_in/ratings_train.txt", sep='\t')
test_tmp = pd.read_csv("KOR_Sentiment-Analysis/data_in/ratings_test.txt", sep='\t')

In [8]:
print(train_data.shape)
print(test_tmp.shape)

(150000, 3)
(50000, 3)


In [9]:
#파일로 저장
train_data.to_csv('train_data.txt', sep = '\t', index = True)
test_tmp.to_csv('test_data.txt', sep = '\t', index = True)

dataset_train= nlp.data.TSVDataset("train_data.txt", field_indices=[2,3], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("test_data.txt", field_indices=[2,3], num_discard_samples=1)

## 3.2 감정사전

In [10]:
dic_data_value = pd.read_table('/content/drive/MyDrive/Colab Notebooks/ML_final/data_in/SentiWord_Dict.txt', header=None)
dic_data_value.rename(columns={dic_data_value.columns[0] : 'document', dic_data_value.columns[1] : 'label'}, inplace=True)
dic_data_value

FileNotFoundError: ignored

In [ ]:
dic_data_value['document'] = dic_data_value['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
dic_data_value['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
dic_data_value = dic_data_value.dropna(how='any') # Null 값 제거
dic_data_value = dic_data_value.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(dic_data_value.isnull().values.any()) # Null 값이 존재하는지 확인
#감성이 없는 단어 삭제
idx_ = dic_data_value[dic_data_value['label']==0.0].index
dic_data_value = dic_data_value.drop(idx_)
print(dic_data_value.groupby(['label']).size().reset_index(name = 'count'))

False
   label  count
0   -2.0   4799
1   -1.0   5016
2    1.0   2246
3    2.0   2603


In [ ]:
# 감정을 숫자로 변환
def emotion_labeling_dic(emotion):
   return{-2.0 : 0, -1.0: 0, 1.0: 1, 2.0: 1}[emotion]

emotion_labels = []

for e in dic_data_value['label']:
   emotion_labels.append(emotion_labeling_dic(e))

dic_data_value['label'] = emotion_labels
dic_data_value

,document,label
10,ㅡㅡ,0
28,ㅅ,0
41,ㄱㅅ,1
42,ㄱㅇㄷ,1
43,가격이 싸다,1
...,...,...
14849,오류,0
14851,의혹,0
14852,내팽개치다,0
14853,횡령,0


In [ ]:
dic_data_value.reset_index(drop=True)

,document,label
0,ㅡㅡ,0
1,ㅅ,0
2,ㄱㅅ,1
3,ㄱㅇㄷ,1
4,가격이 싸다,1
...,...,...
14659,오류,0
14660,의혹,0
14661,내팽개치다,0
14662,횡령,0


In [ ]:
print('전처리 후 사전 샘플의 개수 :',len(dic_data_value))

전처리 후 사전 샘플의 개수 : 14664


## 3.3 훈련용 데이터 합치기

In [ ]:
#합치기 위해 포멧 통일
train_data.drop(labels='id', axis="columns", inplace=True)

In [ ]:
train = pd.concat([dic_data_value, train_data])
#인덱스 재정렬
train.reset_index(drop=True, inplace=True)
train

,document,label
0,ㅡㅡ,0
1,ㅅ,0
2,ㄱㅅ,1
3,ㄱㅇㄷ,1
4,가격이 싸다,1
...,...,...
164659,인간이 문제지.. 소는 뭔죄인가..,0
164660,평점이 너무 낮아서...,1
164661,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
164662,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
#파일로 저장
train.to_csv('train_data.txt', sep = '\t', index = True)
test_tmp.to_csv('test_data.txt', sep = '\t', index = True)

In [16]:
dataset_train= nlp.data.TSVDataset("train_data.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("test_data.txt", field_indices=[2,3], num_discard_samples=1)

## 3.4 테스트 데이터

In [10]:
# 테스트(캐글) 데이터 로드
test_csv = pd.read_csv('KOR_Sentiment-Analysis/data_in/ko_data.csv', encoding = 'cp949')
test_csv.columns = ['id','document']
test_csv.head()

,id,document
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.


In [11]:
test_csv['label'] = 0
test_csv.head(10)

,id,document,label
0,0,정말 많이 울었던 영화입니다.,0
1,1,시간 낭비예요.,0
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.,0
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!,0
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.,0
5,5,잔잔한 감동을 주는 영화가 좋은 영화다. 줄리안 무어의 매력!,0
6,6,"프랑스 영화, 정말 재미없다는 말밖에...",0
7,7,이보다 더 자연스러울 수는 없다.,0
8,8,잠만 자고 있었는데~~,0
9,9,오프닝 씬이... (이 영화와 젊음에 대해 말한다.),0


In [12]:
csv_file = 'KOR_Sentiment-Analysis/data_in/ko_data.csv'
txt_file = 'ko_data.txt'
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r", encoding='cp949') as my_input_file: # , encoding='cp949'
        [ my_output_file.write("\t".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

In [13]:
real_test = nlp.data.TSVDataset("ko_data.txt", field_indices=[1], num_discard_samples=1)

# 모델 구현

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
class BERTDataset_Test(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

In [16]:
## Setting parameters
TEST_SIZE = 0.2
RANDOM_SEED = 42
EPOCHS = 5  # 5

max_len = 128
batch_size = 32
warmup_ratio = 0.1
num_epochs = EPOCHS  # 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [17]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
data_real_test = BERTDataset_Test(real_test, 0, tok, max_len, True, False)

In [18]:
print(len(data_train))
print(len(data_test))
print(len(data_real_test))

150000
50000
11187


In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
real_test_dataloader = torch.utils.data.DataLoader(data_real_test, batch_size=batch_size, num_workers=5)

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
# 디바이스 설정 확인
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [22]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [23]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [24]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [25]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [26]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 모델 학습

In [27]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [28]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    # 모델 학습
    model.train()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # 모델 평가
    model.eval()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


epoch 1 batch id 1 loss 0.7201154232025146 train acc 0.46875
epoch 1 batch id 201 loss 0.6741336584091187 train acc 0.5127487562189055
epoch 1 batch id 401 loss 0.48735085129737854 train acc 0.5729426433915212
epoch 1 batch id 601 loss 0.4599002003669739 train acc 0.6436668053244592
epoch 1 batch id 801 loss 0.4846363365650177 train acc 0.6878511235955056
epoch 1 batch id 1001 loss 0.41947004199028015 train acc 0.7180944055944056
epoch 1 batch id 1201 loss 0.522519052028656 train acc 0.739123646960866
epoch 1 batch id 1401 loss 0.30008700489997864 train acc 0.7554425410421127
epoch 1 batch id 1601 loss 0.5345639586448669 train acc 0.7670596502186133
epoch 1 batch id 1801 loss 0.5351986289024353 train acc 0.7759925041643532
epoch 1 batch id 2001 loss 0.22431674599647522 train acc 0.7836706646676662
epoch 1 batch id 2201 loss 0.6297199726104736 train acc 0.7906065424806906
epoch 1 batch id 2401 loss 0.2813717722892761 train acc 0.7958402748854644
epoch 1 batch id 2601 loss 0.344019979238

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8667626359564939


epoch 2 batch id 1 loss 0.36930051445961 train acc 0.8125
epoch 2 batch id 201 loss 0.27301326394081116 train acc 0.8753109452736318
epoch 2 batch id 401 loss 0.35209113359451294 train acc 0.87803927680798
epoch 2 batch id 601 loss 0.23541902005672455 train acc 0.8794717138103162
epoch 2 batch id 801 loss 0.3374316990375519 train acc 0.8815933208489388
epoch 2 batch id 1001 loss 0.2419770061969757 train acc 0.8827734765234765
epoch 2 batch id 1201 loss 0.48698288202285767 train acc 0.884263114071607
epoch 2 batch id 1401 loss 0.2758365869522095 train acc 0.885639721627409
epoch 2 batch id 1601 loss 0.40661779046058655 train acc 0.8865943160524672
epoch 2 batch id 1801 loss 0.3555607199668884 train acc 0.8881524153248196
epoch 2 batch id 2001 loss 0.3193357288837433 train acc 0.8888055972013993
epoch 2 batch id 2201 loss 0.3060400187969208 train acc 0.8901209677419355
epoch 2 batch id 2401 loss 0.2394755780696869 train acc 0.8911260932944607
epoch 2 batch id 2601 loss 0.0841806009411811

...


epoch 3 test acc 0.8948736404350608


epoch 4 batch id 1 loss 0.3516302704811096 train acc 0.84375
epoch 4 batch id 201 loss 0.17033201456069946 train acc 0.9521144278606966
epoch 4 batch id 401 loss 0.1405288428068161 train acc 0.9541770573566085
epoch 4 batch id 601 loss 0.036561787128448486 train acc 0.9554908485856906
epoch 4 batch id 801 loss 0.15766215324401855 train acc 0.955914481897628
epoch 4 batch id 1001 loss 0.13991796970367432 train acc 0.9562937062937062
epoch 4 batch id 1201 loss 0.48597249388694763 train acc 0.9570410074937552
epoch 4 batch id 1401 loss 0.16487734019756317 train acc 0.9585563882940756
epoch 4 batch id 1601 loss 0.25857865810394287 train acc 0.9593027795128045
epoch 4 batch id 1801 loss 0.19300879538059235 train acc 0.9602998334258745
epoch 4 batch id 2001 loss 0.02889063209295273 train acc 0.9612068965517241
epoch 4 batch id 2201 loss 0.23860661685466766 train acc 0.9617361426624261
epoch 4 batch id 2401 loss 0.01010061800479889 train acc 0.9621902332361516
epoch 4 batch id 2601 loss 0.007


epoch 4 test acc 0.8988323736404351


epoch 5 batch id 1 loss 0.20912492275238037 train acc 0.9375
epoch 5 batch id 201 loss 0.0346820205450058 train acc 0.9693718905472637
epoch 5 batch id 401 loss 0.06700900197029114 train acc 0.9722568578553616
epoch 5 batch id 601 loss 0.030635816976428032 train acc 0.9736896838602329
epoch 5 batch id 801 loss 0.05731307342648506 train acc 0.974289950062422
epoch 5 batch id 1001 loss 0.07278025895357132 train acc 0.9742132867132867
epoch 5 batch id 1201 loss 0.45649945735931396 train acc 0.9747866361365529
epoch 5 batch id 1401 loss 0.15461592376232147 train acc 0.9747947894361171
epoch 5 batch id 1601 loss 0.11311643570661545 train acc 0.9751912866958151
epoch 5 batch id 1801 loss 0.19087892770767212 train acc 0.9756038312048861
epoch 5 batch id 2001 loss 0.0031398036517202854 train acc 0.9761213143428286
epoch 5 batch id 2201 loss 0.18159973621368408 train acc 0.976317582916856
epoch 5 batch id 2401 loss 0.0034070422407239676 train acc 0.9765852769679301
epoch 5 batch id 2601 loss 0.


epoch 5 test acc 0.8972928662827895


In [29]:
test_preds = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(real_test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    out = model(token_ids, valid_length, segment_ids)
    
    for i in range(len(out)):
        test_preds.append(np.argmax(out[i].detach().cpu().numpy()))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
len(test_preds)

11187

In [33]:
# 테스트 데이터의 리뷰 부분을 리스트 처리
test_id = list(test_csv['id'])

# 판다스 데이터프레임 통해 데이터 구성하여 output에 투입
output = pd.DataFrame( data={"Id": test_id, "Predicted": test_preds} )
output.head()

,Id,Predicted
0,0,1
1,1,0
2,2,1
3,3,1
4,4,0


In [34]:
DATA_OUT_NAME = "NSMC_KoBERT_base_MaxLen:"+ str(max_len) + "_BchSize:"+ str(batch_size) + ".csv"

# csv파일 생성
output.to_csv(DATA_OUT_NAME, index = False)  # 앙상블 조합 및 캐글 제출 용도

from google.colab import files

files.download(DATA_OUT_NAME)
#0.90041

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>